# Downloading MODIS MCD43A3 broadband albedo

## Download data

First we have to import pymodis library and set all the needed variables

In [1]:
import os
import sys
import glob
from pathlib import Path
from IPython.core.display import Image
from pymodis import downmodis
from pymodis import parsemodis
from pymodis.convertmodis_gdal import convertModisGDAL
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tifffile as tiff

# Your Earthdata login info
Earthdata_user = ""
Earthdata_password ""

In [2]:
# destination folder for raw HDF files
dest = "/home/cluster/eplekh/data/modis/raw/2000/"
Path(dest).mkdir(parents=True, exist_ok=True)

# output destination folder for tiff mosaics
output_dest = "/home/cluster/eplekh/data/modis/mosaics/"
Path(output_dest).mkdir(parents=True, exist_ok=True)

# tiles to download - all the tiles belonging to Arctic
tiles = ["h09v02"] + ["h" + str(i) + "v02" for i in range(10,27)] + \
["h" + str(i) + "v01" for i in range(12,24)] + ["h" + str(i) + "v00" for i in range(15,21)]
# number of day to download
delta = 1

### Download HDFs and mosaic them

In [3]:
from pymodis.convertmodis_gdal import createMosaicGDAL
# selecting the bands to download from the MCD43A3 product. Thse are: shortwave, vis and nir+swir broadband albedos
subset = [0 for i in range(29)]
subset[27:30] = [1,1,1]


<div style="border: 1px solid red;"><h6 style="color:#ff4040; margin: 0.5em;">Warning</h6>
<span style="margin: 0.5em;">The next operation could take some minutes depending on your internet speed</span></div>

In [7]:
%%time 
for year in range(2000, 2022): 
    print(year)
    #selecting 15th of July each year
    day = str(year) + "-07-15"
    
    #preparing the output desination
    dest = "/home/cluster/eplekh/data/modis/raw/"+ str(year) + "/"
    Path(dest).mkdir(parents=True, exist_ok=True)
    output_pref = os.path.join(output_dest, str(year) + '.mosaic')
    
    #initializing the downmodis object.
    modisDown = downmodis.downModis(destinationFolder=dest, tiles=tiles, today=day, delta=delta,
                               product='MCD43A3.061', path='MOTA', user = Earthdata_user, password = Earthdata_password)
    %time modisDown.connect()
    %time modisDown.downloadsAllDay()
    files = glob.glob(os.path.join(dest, 'MCD43A3*.hdf'))
    print(len(files))
    
    %time mosaic = createMosaicGDAL(files, subset, 'GTiff')
    mosaic.write_vrt(output_pref)

2000
CPU times: user 194 ms, sys: 2.94 ms, total: 197 ms
Wall time: 20 s
CPU times: user 3.26 s, sys: 8.69 s, total: 11.9 s
Wall time: 6min
34
CPU times: user 649 ms, sys: 7.71 s, total: 8.36 s
Wall time: 8.38 s
The VRT mosaic file /home/cluster/eplekh/data/modis/mosaics/2000.mosaic has been created
2001
CPU times: user 195 ms, sys: 5.62 ms, total: 201 ms
Wall time: 6.59 s
CPU times: user 3.13 s, sys: 9.13 s, total: 12.3 s
Wall time: 4min 59s
34
CPU times: user 720 ms, sys: 7.02 s, total: 7.74 s
Wall time: 7.79 s
The VRT mosaic file /home/cluster/eplekh/data/modis/mosaics/2001.mosaic has been created
2002
CPU times: user 191 ms, sys: 8.9 ms, total: 200 ms
Wall time: 5.75 s
CPU times: user 3.04 s, sys: 9.05 s, total: 12.1 s
Wall time: 5min 24s
34
CPU times: user 658 ms, sys: 8.09 s, total: 8.75 s
Wall time: 8.79 s
The VRT mosaic file /home/cluster/eplekh/data/modis/mosaics/2002.mosaic has been created
2003
CPU times: user 197 ms, sys: 0 ns, total: 197 ms
Wall time: 7.1 s
CPU times: user

### Reproject all mosaics

In [ ]:
%%time 
vrtfiles = glob.glob(os.path.join(output_dest, '*.vrt'))
from pymodis.convertmodis_gdal import convertModisGDAL
wkt = "+proj=laea +lat_0=90 +lon_0=180 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

for f in vrtfiles:
    base = os.path.basename(f).replace('.vrt', '_vrt')
    output = os.path.join(output_dest, base)
    convertsingle = convertModisGDAL(hdfname=f, prefix=output, subset= subset, res=500, wkt = wkt, vrt=True)
    convertsingle.run_vrt_separated() 

### Crop all files to CAVM

In [9]:
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import mapping

In [10]:
cavm_path = "/home/cluster/eplekh/data/cavm/shape/cp_veg_la.shp" # this is CAVM shapefile
crop_extent = gpd.read_file(cavm_path)

In [12]:
clip_path = "/home/cluster/eplekh/data/modis/cropped_mosaics/"
Path(clip_path).mkdir(parents=True, exist_ok=True)
tiffiles = glob.glob(os.path.join(output_dest, '*shortwave_vrt.tif'))
len(tiffiles)

22

In [14]:
# Crop mosaics for shortwave, vis and nir+swir bands
for broadband in ["shortwave", "vis", "nir"]:
    clip_path = "/home/cluster/eplekh/data/modis/cropped_mosaics_" + broadband + "/"
    Path(clip_path).mkdir(parents=True, exist_ok=True)
    for year in range(2000, 2022):
        print(year)
        tif_path = output_dest+str(year)+".mosaic_Albedo_WSA_" + broadband + "_vrt.tif"
        modis_year = rxr.open_rasterio(tif_path, masked=True).squeeze()
        %time modis_year_clipped = modis_year.rio.clip(crop_extent.geometry.apply(mapping), crop_extent.crs)
        # Write the data to a new geotiff file
        modis_year_clipped.rio.to_raster(clip_path + str(year) + "_" + broadband + ".tif")

2000
CPU times: user 2.85 s, sys: 3.12 s, total: 5.97 s
Wall time: 5.97 s
2001
CPU times: user 2.82 s, sys: 789 ms, total: 3.61 s
Wall time: 3.97 s
2002
CPU times: user 2.86 s, sys: 844 ms, total: 3.71 s
Wall time: 4.09 s
2003
CPU times: user 2.86 s, sys: 828 ms, total: 3.69 s
Wall time: 4.08 s
2004
CPU times: user 2.8 s, sys: 848 ms, total: 3.65 s
Wall time: 3.94 s
2005
CPU times: user 2.88 s, sys: 855 ms, total: 3.74 s
Wall time: 4.42 s
2006
CPU times: user 2.72 s, sys: 20.5 s, total: 23.2 s
Wall time: 24.6 s
2007
CPU times: user 2.68 s, sys: 20.7 s, total: 23.4 s
Wall time: 24.8 s
2008
CPU times: user 2.93 s, sys: 972 ms, total: 3.9 s
Wall time: 4.23 s
2009
CPU times: user 2.9 s, sys: 884 ms, total: 3.78 s
Wall time: 4.08 s
2010
CPU times: user 2.76 s, sys: 895 ms, total: 3.66 s
Wall time: 3.87 s
2011
CPU times: user 2.86 s, sys: 901 ms, total: 3.76 s
Wall time: 3.99 s
2012
CPU times: user 2.78 s, sys: 833 ms, total: 3.61 s
Wall time: 3.84 s
2013
CPU times: user 2.86 s, sys: 886 ms,